<a href="https://colab.research.google.com/github/boiBASH/Tolaram_Project/blob/main/ML_Model_for_Customer_Profilling_unmasked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import pandas
import pandas as pd

# Step 3: Load your dataset (replace with the actual path in your Drive)
file_path = "/content/drive/MyDrive/TOLARAM/Data sample analysis.csv"
df = pd.read_csv(file_path, encoding='latin-1')

Mounted at /content/drive


<ipython-input-1-795eecdcd5c0>:18: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='latin-1')


In [2]:
# Step 1: Convert Delivered_date to datetime
df['Delivered_date'] = pd.to_datetime(df['Delivered_date'], errors='coerce', dayfirst=True)
df

,Brand,Branch,Designation,SKU_Code,Salesman_Code,Salesman_Name,Customer_Name,Customer_Phone,Order_Id,Delivered_date,Delivered Qty,Redistribution Value
0,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807259,ANOSIKE MIRACLE CHIMECHEFULAM,Eze store,8060733751,11432770,2024-08-05,3.0,"46,500"
1,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Chimaka,9016204667,11565436,2024-08-26,1.0,"15,500"
2,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Chimaka,9016204667,11727113,2024-09-17,1.0,"15,500"
3,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Cynthia chi,8060359775,11756045,2024-09-20,1.0,"15,500"
4,ADDMIE,LAGOS 1,FIELD EXECUTIVE,10002778,807881,OKERE IFEANYI,Daddy Josephine,9075013565,11585122,2024-08-29,1.0,"15,500"
...,...,...,...,...,...,...,...,...,...,...,...,...
108820,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,House 1 super store,8101599308,12090661,2024-11-01,1.0,"56,800"
108821,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,Loveth ejiro,8139407516,12282897,2024-11-22,1.0,"33,000"
108822,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,Mrs Jennifer,8060468927,12067581,2024-10-30,4.0,"201,600"
108823,POWER OIL,LAGOS 1,VAN SALES EXECUTIVE,VOB4500_,810667,ONYIA CHUKA,TWINNY SQUARE ENTERPRISES,8055148535,12032444,2024-10-25,1.0,"50,400"


In [6]:
df = df.rename(columns={
    'Delivered_date': 'delivered_date',
    'Delivered Qty': 'delivered_qty',
    'Redistribution Value': 'spend'
})

# Convert date column (Based on your sample '2024-08-05', Year-Month-Day)
df['delivered_date'] = pd.to_datetime(df['delivered_date'], errors='coerce', dayfirst=False)

# Clean and convert spend column
df['spend'] = df['spend'].astype(str).str.replace(',', '', regex=False).astype(float)

# Filter and sort data (Sorting by Customer_Phone, SKU_Code, Brand, and date)
# Including Brand in sorting can help with subsequent grouped operations if needed.
df = df[df['delivered_qty'].astype(float) > 0].sort_values(['Customer_Phone', 'SKU_Code', 'Brand', 'delivered_date'])

# Step 4: Feature Engineering (Now grouping by Customer_Phone, SKU_Code, AND Brand)
# This will calculate features at a more granular level: for each customer, SKU, and brand combination.
df['prev_date'] = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand'])['delivered_date'].shift(1) # <--- Added Brand to groupby
df['recency_days'] = (df['delivered_date'] - df['prev_date']).dt.days.fillna(-1)
df['cum_freq'] = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand']).cumcount() # <--- Added Brand to groupby
df['cum_spend'] = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand'])['spend'].cumsum() - df['spend'] # <--- Added Brand to groupby
df['avg_spend_past'] = df['cum_spend'] / df['cum_freq'].replace(0, 1)
df['cum_qty'] = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand'])['delivered_qty'].cumsum() - df['delivered_qty'] # <--- Added Brand to groupby
df['avg_qty_past'] = df['cum_qty'] / df['cum_freq'].replace(0, 1)

# Step 5: Compute Targets (Grouping by Customer_Phone, SKU_Code, AND Brand)
# Targets are computed at the same granular level.
df['next_date'] = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand'])['delivered_date'].shift(-1) # <--- Added Brand to groupby
df['next_days'] = (df['next_date'] - df['delivered_date']).dt.days
df['next_spend'] = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand'])['spend'].shift(-1) # <--- Added Brand to groupby
df['next_qty'] = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand'])['delivered_qty'].shift(-1) # <--- Added Brand to groupby
df['y_buy'] = df['next_days'].notnull().astype(int)

# Step 6: Encode SKU and Brand
le_sku = LabelEncoder().fit(df['SKU_Code'])
df['sku_enc'] = le_sku.transform(df['SKU_Code'])

le_brand = LabelEncoder().fit(df['Brand']) # <--- Encode Brand
df['brand_enc'] = le_brand.transform(df['Brand']) # <--- Create encoded Brand column

# We will not encode Customer_Name directly as a feature in this core model
# because there are likely many unique customer names, which would make encoding difficult
# and potentially less useful for a standard classification/regression model.
# Customer_Phone is the primary identifier.

# Step 7: Define Features (Including the encoded Brand)
features = ['recency_days', 'cum_freq', 'avg_spend_past', 'avg_qty_past', 'sku_enc', 'brand_enc'] # <--- Added brand_enc

# Step 8: Train Models (Keeping the same models and approach, using new features)
X_clf = df[features]
y_clf = df['y_buy']
Xc_train, Xc_test, yc_train, yc_test = train_test_split(X_clf, y_clf, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=20, random_state=42).fit(Xc_train, yc_train)

# Train regressors on positive instances (Keep the same approach)
df_reg = df[df['y_buy'] == 1].dropna(subset=['next_days', 'next_spend', 'next_qty'])
X_reg = df_reg[features] # <--- Using updated features
y_days = df_reg['next_days']
y_spend = df_reg['next_spend']
y_qty = df_reg['next_qty']

reg_days = RandomForestRegressor(n_estimators=20, random_state=42).fit(X_reg, y_days)
reg_spend = RandomForestRegressor(n_estimators=20, random_state=42).fit(X_reg, y_spend)
reg_qty = RandomForestRegressor(n_estimators=20, random_state=42).fit(X_reg, y_qty)

# Step 9: Predict on latest snapshot (Now grouping by Customer_Phone, SKU_Code, and Brand)
# We'll group by Customer_Phone, SKU_Code, and Brand to get the last transaction details
# for each unique combination of these.
latest = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand']).last().reset_index() # <--- Added Brand to groupby

# Calculate recency from the current date
latest['last_purchase_date'] = latest['delivered_date']
latest['recency_days'] = (pd.Timestamp.now() - latest['delivered_date']).dt.days

# Ensure the encoded SKU and Brand are present in the latest data
latest['sku_enc'] = le_sku.transform(latest['SKU_Code'])
latest['brand_enc'] = le_brand.transform(latest['Brand']) # <--- Added encoded Brand

# Ensure other features used in prediction are available in the 'latest' DataFrame
# These are already calculated in the feature engineering step and carried over by .last()
latest['cum_freq'] = latest['cum_freq']
latest['avg_spend_past'] = latest['avg_spend_past']
latest['avg_qty_past'] = latest['avg_qty_past']

# Select features for prediction
X_pred = latest[features] # <--- Using updated features

# Make predictions
latest['probability'] = clf.predict_proba(X_pred)[:, 1]
latest['pred_days'] = reg_days.predict(X_pred)
latest['pred_next_date'] = latest['last_purchase_date'] + pd.to_timedelta(latest['pred_days'], unit='D')
latest['pred_spend'] = reg_spend.predict(X_pred)
latest['pred_qty'] = reg_qty.predict(X_pred)

# Step 10: Extract and Display Results (Adjusting display to include Brand)
# We'll sort by Customer_Phone and probability to show top predictions per customer.
# The results will now be for each Customer_Phone, SKU_Code, and Brand combination.
results = (latest.sort_values(['Customer_Phone', 'probability'], ascending=[True, False])
                  .groupby('Customer_Phone').head(3).reset_index(drop=True))

# Display the relevant prediction columns (Including Brand)
pred_df = results[['Customer_Phone', 'SKU_Code', 'Brand', 'last_purchase_date', # <--- Added Brand
                   'pred_next_date', 'pred_spend', 'pred_qty', 'probability']]
pred_df

,Customer_Phone,SKU_Code,Brand,last_purchase_date,pred_next_date,pred_spend,pred_qty,probability
0,7010009941,10002849,MUNCH IT,2024-10-03,2024-10-15 06:00:00.000000000,20402.500000,1.000000,0.000000
1,7010009941,10002857,MUNCH IT,2025-01-02,2025-01-27 07:12:00.000000000,20550.000000,1.000000,0.000000
2,7010009941,10002866,MUNCH IT,2024-12-26,2025-01-14 04:48:00.000000000,20150.000000,1.000000,0.000000
3,7010147753,10002835,KELLOGGS,2024-09-09,2024-10-09 23:33:16.230299469,36887.303805,1.283440,0.293582
4,7010147753,10002957,COLGATE,2024-11-28,2024-12-15 13:56:01.363753769,186899.389908,1.697890,0.167054
...,...,...,...,...,...,...,...,...
11382,9167401267,10002830,KELLOGGS,2024-10-22,2024-11-24 14:03:20.871039798,42176.252494,1.181941,0.167916
11383,9167401267,10003376,KELLOGGS,2025-01-07,2025-01-31 00:47:10.697095639,20405.641184,1.280265,0.153322
11384,9169810564,10000533,POWER OIL,2024-11-21,2024-12-23 03:17:29.587539811,76060.624893,9.275119,0.256871
11385,9169810564,10003395,KELLOGGS,2025-01-21,2025-02-16 00:19:11.809279920,34677.497233,1.566237,0.215214


In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    roc_auc_score, classification_report,
    mean_absolute_error, mean_squared_error, r2_score
)

# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import pandas
import pandas as pd

# Step 3: Load your dataset (replace with the actual path in your Drive)
file_path = "/content/drive/MyDrive/TOLARAM/Data sample analysis.csv"
df = pd.read_csv(file_path, encoding='latin-1')

df = df.rename(columns={
    'Delivered_date': 'delivered_date',
    'Delivered Qty': 'delivered_qty',
    'Redistribution Value': 'spend'
})
df['delivered_date'] = pd.to_datetime(df['delivered_date'], errors='coerce', dayfirst=True)
df['spend'] = df['spend'].str.replace(',', '').astype(float)
df = df[df['delivered_qty'] > 0] \
       .sort_values(['Customer_Phone','SKU_Code','delivered_date'])

# 2) Features & targets (Now grouping by Customer_Phone, SKU_Code, AND Brand)
df['prev_date'] = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand'])['delivered_date'].shift(1) # <--- Added Brand to groupby
df['recency_days'] = (df['delivered_date'] - df['prev_date']).dt.days.fillna(-1)
df['cum_freq']      = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand']).cumcount() # <--- Added Brand to groupby
df['cum_spend']     = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand'])['spend'].cumsum() - df['spend'] # <--- Added Brand to groupby
df['avg_spend_past']= df['cum_spend'] / df['cum_freq'].replace(0,1)
df['cum_qty']       = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand'])['delivered_qty'].cumsum() - df['delivered_qty'] # <--- Added Brand to groupby
df['avg_qty_past']  = df['cum_qty'] / df['cum_freq'].replace(0,1)
df['next_date']  = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand'])['delivered_date'].shift(-1) # <--- Added Brand to groupby
df['next_days']  = (df['next_date'] - df['delivered_date']).dt.days
df['next_spend']= df.groupby(['Customer_Phone', 'SKU_Code', 'Brand'])['spend'].shift(-1) # <--- Added Brand to groupby
df['next_qty']  = df.groupby(['Customer_Phone', 'SKU_Code', 'Brand'])['delivered_qty'].shift(-1) # <--- Added Brand to groupby
df['y_buy']     = df['next_days'].notnull().astype(int)

# Encode SKU and Brand
le_sku = LabelEncoder().fit(df['SKU_Code'])
df['sku_enc'] = le_sku.transform(df['SKU_Code'])

le_brand = LabelEncoder().fit(df['Brand']) # <--- Encode Brand
df['brand_enc'] = le_brand.transform(df['Brand']) # <--- Create encoded Brand column

# Define features (Including the encoded Brand)
features = ['recency_days', 'cum_freq', 'avg_spend_past', 'avg_qty_past', 'sku_enc', 'brand_enc'] # <--- Added brand_enc

X_clf = df[features]
y_clf = df['y_buy']
Xc_train, Xc_test, yc_train, yc_test = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42
)
clf = RandomForestClassifier(n_estimators=20, random_state=42)
clf.fit(Xc_train, yc_train)

# --- Evaluation requires importing metrics ---
from sklearn.metrics import roc_auc_score, classification_report

probs = clf.predict_proba(Xc_test)[:,1]
preds = clf.predict(Xc_test)

print("=== CLASSIFIER METRICS ===")
print("ROC-AUC:", roc_auc_score(yc_test, probs))
print(classification_report(yc_test, preds))

# Regressor setup (features include brand_enc)
df_reg = df[df['y_buy']==1].dropna(subset=['next_days','next_spend','next_qty'])
Xr = df_reg[features] # <--- Xr uses features including brand_enc
yd = df_reg['next_days']
ys = df_reg['next_spend']
yq = df_reg['next_qty']

# Train-test split for regressors
Xr_train, Xr_test, yd_train, yd_test, ys_train, ys_test, yq_train, yq_test = train_test_split(
    Xr, yd, ys, yq, test_size=0.2, random_state=42
)

# Train regressors
reg_days  = RandomForestRegressor(n_estimators=20, random_state=42).fit(Xr_train, yd_train)
reg_spend = RandomForestRegressor(n_estimators=20, random_state=42).fit(Xr_train, ys_train)
reg_qty   = RandomForestRegressor(n_estimators=20, random_state=42).fit(Xr_train, yq_train)

print("\n=== REGRESSOR METRICS ===")
for name, y_true, model in [
    ("Next-Days", yd_test, reg_days),
    ("Spend",     ys_test, reg_spend),
    ("Quantity",  yq_test, reg_qty)
]:
    y_pred = model.predict(Xr_test)
    print(f"\n-- {name} --")
    print(" MAE :", mean_absolute_error(y_true, y_pred))
    print(" RMSE:", np.sqrt(mean_squared_error(y_true, y_pred)))
    print(" R2  :", r2_score(y_true, y_pred))


pred_df = results[['Customer_Phone', 'SKU_Code', 'Brand', 'last_purchase_date', # Added Brand
                  'pred_next_date', 'pred_spend', 'pred_qty', 'probability']]
pred_df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-9-8b660b94899f>:21: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='latin-1')


=== CLASSIFIER METRICS ===
ROC-AUC: 0.7595757018329582
              precision    recall  f1-score   support

           0       0.68      0.67      0.67      9136
           1       0.72      0.73      0.72     10648

    accuracy                           0.70     19784
   macro avg       0.70      0.70      0.70     19784
weighted avg       0.70      0.70      0.70     19784


=== REGRESSOR METRICS ===

-- Next-Days --
 MAE : 17.42809909595842
 RMSE: 24.22985137658936
 R2  : 0.06876770341719929

-- Spend --
 MAE : 35453.55440328573
 RMSE: 113395.89584650511
 R2  : 0.09641035167389167

-- Quantity --
 MAE : 2.5521819120937557
 RMSE: 8.523112907949033
 R2  : 0.13561280774841722
